##### 분류<br><br>ImageNet Database에는 1,400만장의 이미지가 20,000개 이상의 범주로 분류돼 있다. 각 사진에는 명확한 하나의 범주가 정해져 있는 것이다.<br>ImageNet Database중 일부를 사용해 이미지의 범주를 분류하는 대회인 ILSVRC에서는 2012년 딥러닝 기반의 CNN이 등장한 이후 정확도가 크게 상승했다. 2017년에는 거의 100%에 가까운 정확도를 달성했기 때문에 이 대회는 더 이상 열리지 않고 있다.<br><br>분류 문제에는 명확한 정답이 있는 경우도 있지만 그렇지 않은 경우도 있다. 예를 들어 개인정보가 가려진 신용카드 사용 정보로 비슷한 유형의 구매자 군집을 분류하는 문제도 분류 문제의 일종이다. 이것은 정답이 없기 때문에 비지도학습(Unsupervised Learning)이라고 불린다. 9장의 AutoEncoder에서 이에 관한 좀 더 자세한 내용을 다룰 것이다. 